# Relu and Intutition
Source: https://gist.github.com/alekseynp/41f70c509176cbff765e79ebe5edaa6a

### Preliminaries

In [ ]:
# Import modules
import numpy as np
np.random.seed(0)

import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Model 
from keras.layers import Input, Dense
import keras.optimizers

### Scaffolding

In [ ]:
batch_size = 32
plot_x = np.arange(-1.0, 1.0, 0.01)

### Functions

In [ ]:
def f(x):
    if x < 0.0:
        return 0.0
    else:
        return x

In [ ]:
def get_model(hidden_units):
    model_input = Input(shape=(1,), dtype='float32', name='input')
    net = Dense(hidden_units, activation='relu')(model_input)
    out = Dense(1, activation='linear')(net)

    model = Model(inputs=model_input, outputs=out)
    optimizer = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.0)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    
    return model

In [ ]:
def poly_generator(batch_size,coefficients):
    output_x = np.zeros((batch_size))
    output_y = np.zeros((batch_size))
       
    while True:
        output_x = np.random.uniform(-1.0,1.0,batch_size)
        output_y = coefficients[0] = sum([ coefficients[i+1] * np.power(output_x, i+1) for i in range(len(coefficients)-1)])

        yield output_x, output_y

In [ ]:

def plot_poly(coefficients, ax):
    plot_y = coefficients[0] = sum([ coefficients[i+1] * np.power(plot_x, i+1) for i in range(len(coefficients)-1)])
    ax.plot(plot_x, plot_y, lw=2)

In [ ]:

def plot_model(model, ax):
    plot_y = model.predict(plot_x)
    ax.plot(plot_x, plot_y)

In [ ]:
def attempt(hidden_units, coefficients, ax):
    model = get_model(hidden_units)
    model.fit_generator(poly_generator(batch_size, coefficients),steps_per_epoch=10000,epochs=1,verbose=0)
    plot_model(model, ax)


In [ ]:
def run_coefficients(coefficients):
    fig, axarr = plt.subplots(3, 3, figsize=(9,9))

    for i, ax in enumerate(np.ravel(axarr)):

        plot_poly(coefficients, ax)
        attempt(i+1,coefficients, ax)
        ax.set_title('Hidden Units: {}'.format(i+1))
        print("Done:",i)

    plt.tight_layout()

## Experiments
Target function: $$f(x) = 0.1 + 0.2x + 0.3x^2 + 0.4x^3$$
### V1
Single layer neural network with between 1 and 9 hidden units

In [ ]:
run_coefficients([0.1,0.2,0.3,0.4])

### V2
Target Functions: $$f(x) = x^2$$

In [ ]:
run_coefficients([0,0,1])

### V3
Target Function: $$f(x) = 0.1 + 0.2x + 0.3x^2 + 0.4x^3 + 0.5x^4$$

## Dig a Little Deeper
An interesting experiment above that might be simple enough to gain some inuition is the 4 neuron network attempting to fit

$$f(x) = x^2$$

In [ ]:
coefficients = [0,0,1]
model = get_model(4)
model.summary()
model.fit_generator(poly_generator(batch_size, coefficients),
                   steps_per_epoch = 10000, epochs=1, verbose=1)